## Imports

In [ ]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

from astropy.timeseries import LombScargle
%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 20})

## Load Lines and Data

In [ ]:
data_path = '../data'
lines_vac, lines_air = np.loadtxt(f'{data_path}/hydroxyl_lines_rousselot_2000.txt').T

In [ ]:
results_path = '../results'
df = pd.read_csv(f'{results_path}/lines_norm_gdl_fit.csv', index_col=0)
columns = df.columns

In [ ]:
time_centered = df['spec_mjd'] - int(df['spec_mjd'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

In [ ]:
df_col_line_sum = [f'{line}_sum' for line in lines_vac]

In [ ]:
df_line_sums = df[df_col_line_sum][mask1]

In [ ]:
time_centered_mask1 = time_centered[mask1].values
flux_median_mask1 = df_line_sums.median(axis=1).values

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Total Flux Contribution (Event 1; Median)', fontsize=fs)
plt.scatter(time_centered_mask1, flux_median_mask1, color='C5', alpha=0.5, label='5 minute cadence')
plt.scatter(time_centered_mask1[::5], flux_median_mask1[::5], color='C9', label='25 minute cadence')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)

## LSP

In [ ]:
min_freq = 1 / 365
max_freq = 10 * 24

**For each iteration, remove up to half the data points from the median flux time series, perform LSP, and record the most dominant period.**

In [ ]:
lst_periods = []
length_arrays = []
for i in tqdm(range(10000)):
    remove_ind = np.random.randint(0, flux_median_mask1.shape[0], flux_median_mask1.shape[0]//2)
    t_use = np.delete(time_centered_mask1, remove_ind)
    y_use = np.delete(flux_median_mask1, remove_ind)
    length_arrays.append(y_use.shape[0])
    frequency, power = LombScargle(t_use, y_use).autopower(minimum_frequency=min_freq, maximum_frequency=max_freq)
    p = 1/frequency[np.argmax(power)]
    lst_periods.append(p)
lst_periods = np.array(lst_periods)
length_arrays = np.array(length_arrays)

In [ ]:
plt.hist(lst_periods, bins=100)
np.mean(lst_periods), np.median(lst_periods)
plt.xlabel('Period (days)')
plt.yscale('log')

In [ ]:
true_period = np.median(lst_periods)

In [ ]:
plt.hist(length_arrays, bins=20)
plt.xlabel('Number of data points')
plt.show()

**Find the period's relative error between using a different cadence.**

In [ ]:
frequency, power = LombScargle(time_centered_mask1, flux_median_mask1).autopower(minimum_frequency=min_freq, maximum_frequency=max_freq)

In [ ]:
f = frequency[np.argmax(power)]
f

In [ ]:
period_true = 1/f

In [ ]:
period_true

In [ ]:
frequency, power = LombScargle(time_centered_mask1[::5], flux_median_mask1[::5]).autopower(minimum_frequency=min_freq, maximum_frequency=max_freq)

In [ ]:
period_25 = 1 / frequency[np.argmax(power)]

In [ ]:
period_25

In [ ]:
np.abs(period_25 - period_true) / period_true * 100

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter((time_centered_mask1%period_25)/period_25, flux_median_mask1, s=1)
plt.xlabel('Phase')
plt.ylabel('Relative Flux')
plt.title('Phase folding on Event')

**For each iteration, remove an increasing number of data points (i.e. first remove 0 data points, then 1, then 2, etc.) 100 times and record the mean/median of the most dominant period.**

In [ ]:
lst_period_stats = []
for i in tqdm(range(flux_median_mask1.shape[0])):
    lst_period_stats_j = []
    for j in range (100):
        remove_ind = np.random.choice(flux_median_mask1.shape[0], i, replace=False)
        t_use = np.delete(time_centered_mask1, remove_ind)
        y_use = np.delete(flux_median_mask1, remove_ind)
        frequency, power = LombScargle(t_use, y_use).autopower(minimum_frequency=min_freq, maximum_frequency=max_freq)
        p = 1/frequency[np.argmax(power)]
        lst_period_stats_j.append(p)
    lst_period_stats_i = [np.mean(np.sort(lst_period_stats_j)[10:90]), 
                          np.median(lst_period_stats_j), 
                          np.nanstd(np.sort(lst_period_stats_j)[10:90]), 
                          median_abs_deviation(lst_period_stats_j)]
    lst_period_stats.append(lst_period_stats_i)
lst_period_stats = np.array(lst_period_stats)

In [ ]:
percentage = 1 - (np.arange(flux_median_mask1.shape[0]) / flux_median_mask1.shape[0])

In [ ]:
plt.figure(figsize=[30,10])
plt.errorbar(percentage, lst_period_stats[:, 0], lst_period_stats[:, 2], fmt='o')
plt.hlines(period_true, -0.1, 1.1, color='C1')
plt.title('LSP Stability', fontsize=30)
plt.xlabel('Percentage of Data Kept')
plt.ylabel('')
plt.yscale('log')

In [ ]:
fs=45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.errorbar(percentage*100, lst_period_stats[:, 1], lst_period_stats[:, 3], fmt='o', label='experiment')
plt.hlines(period_true, -10, 110, color='C1', label='true period')
plt.title('SPIRou Sky: LSP Stability (Event 1)', fontsize=fs)
plt.xlabel('Percentage of Data Kept', fontsize=fs)
plt.ylabel('Median Period in Days \n(100 experiments)', fontsize=fs)
plt.legend(fontsize=fs)
plt.tick_params(labelsize=fs)